In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# 資料預處理
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))])

In [2]:
# 分割training和valid的資料
train_dataset, valid_dataset = torch.utils.data.random_split(
    torchvision.datasets.CIFAR100(root="./data", train=True, download=True, transform=transform)
    , lengths=[40000, 10000])

#拿取load的資料
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
vaildloader = torch.utils.data.DataLoader(valid_dataset, batch_size=64, shuffle=True, num_workers=2)
test_dataset = torchvision.datasets.CIFAR100(root="./data", train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=2)

classes = ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm']

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# 建構block
class ResidualBlock(nn.Module):
  def __init__(self, in_channel, out_channel, strides=1):
    super(ResidualBlock, self).__init__()
    self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=strides, padding=1)
    self.bn1 = nn.BatchNorm2d(out_channel)
    self.relu = nn.ReLU(inplace=True)
    self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, stride=1, padding=1)
    self.bn2 = nn.BatchNorm2d(out_channel)

    self.downsample = None
    if strides != 1 or in_channel != out_channel:
        self.downsample = nn.Sequential(
        nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=strides),
        nn.BatchNorm2d(out_channel)
    )

  def forward(self, x):
    identity = x

    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)
    out = self.conv2(out)
    out = self.bn2(out)

    if self.downsample is not None:
      identity = self.downsample(x)

    out += identity
    out = self.relu(out)

    return out

#建構Resnet
class ResNet18(nn.Module):
  def __init__(self, ResidualBlock, num_class=100):
    super(ResNet18, self).__init__()

    # 第一層
    self.channel = 64
    self.conv1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
                               nn.BatchNorm2d(64),
                               nn.ReLU(),
                               nn.MaxPool2d(kernel_size=3, stride=2))
    # 第2~5層
    self.layer1 = self.make_layer(ResidualBlock, 64, 2, 1)
    self.layer2 = self.make_layer(ResidualBlock, 128, 2, 2)
    self.layer3 = self.make_layer(ResidualBlock, 256, 2, 2)
    self.layer4 = self.make_layer(ResidualBlock, 512, 2, 2)
    self.avgpool = nn.AdaptiveAvgPool2d((1,1)) #平均池化層
    self.fcl = nn.Linear(512, num_class) #全連通層

  # 建立每一層
  def make_layer(self, block, outchannel, num_block, stride):
    strides = [stride] + [1] * (num_block - 1) #只有第一層stride會不一定是，剩下都是1
    layer = []

    for i in strides:
      layer.append(block(self.channel, outchannel, i))
      self.channel = outchannel

    return nn.Sequential(*layer)

  def forward(self, x):
    out = self.conv1(x)
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = self.avgpool(out)

    out = torch.flatten(out, 1)
    out = self.fcl(out)   

    return out

In [4]:
# 建立model
model = ResNet18(ResidualBlock)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet18(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kern

In [17]:
# 損失函數、優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00003)

from tqdm.notebook import tqdm

# 開始訓練
for epoch in range(10):
  model.train()
  running_loss = 0.0
  progress_bar = tqdm(enumerate(trainloader, 0), total=len(trainloader), desc="Training")

  for inputs, labels in trainloader:
    inputs, labels = inputs.to(device), labels.to(device)
    
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    progress_bar.update(1)
    progress_bar.set_description(f"Epoch{epoch + 1}, Loss: {running_loss / len(trainloader):.4f}")

#vaild data
model.eval()
with torch.no_grad():
  correct = 0
  total = 0
  for inputs, labels in vaildloader:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct = (predicted == labels).sum().item()
print("Finished training")

Training:   0%|          | 0/625 [00:00<?, ?it/s]

Training:   0%|          | 0/625 [00:00<?, ?it/s]

Training:   0%|          | 0/625 [00:00<?, ?it/s]

Training:   0%|          | 0/625 [01:20<?, ?it/s]

Training:   0%|          | 0/625 [00:00<?, ?it/s]

...

Training:   0%|          | 0/625 [00:00<?, ?it/s]

Training:   0%|          | 0/625 [00:00<?, ?it/s]

Training:   0%|          | 0/625 [00:00<?, ?it/s]

Training:   0%|          | 0/625 [00:00<?, ?it/s]

Finished training


In [18]:
# 把訓練好的模型，用test data看看結果精不精確
correct = 0
total = 0
with torch.no_grad():
  for data in testloader:
    inputs, labels = data[0].to(device), data[1].to(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    correct += (predicted == labels).sum().item()
    total += labels.size(0)

print(f"Accuracy of the network on CIFAR100: {100 * correct / total}%")
print()

class_correct = list(0. for i in range(100))
class_total = list(0. for i in range(100))

with torch.no_grad():
  for data in testloader:
    inputs, labels = data[0].to(device), data[1].to(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).squeeze()
    
    for i in range(len(labels)):
      label = labels[i]
      class_correct[label] += correct[i].item()
      class_total[label] += 1

print("-------------------------------------------")
for i in range(100):
  print(f"Accuracy of {classes[i]} : {100 * class_correct[i]/class_total[i]}%")


Accuracy of the network on CIFAR100: 46.05%

-------------------------------------------
Accuracy of apple : 74.0%
Accuracy of aquarium_fish : 57.0%
Accuracy of baby : 28.0%
Accuracy of bear : 26.0%
Accuracy of beaver : 25.0%
Accuracy of bed : 45.0%
Accuracy of bee : 56.0%
Accuracy of beetle : 51.0%
Accuracy of bicycle : 55.0%
Accuracy of bottle : 58.0%
Accuracy of bowl : 30.0%
Accuracy of boy : 30.0%
Accuracy of bridge : 48.0%
Accuracy of bus : 35.0%
Accuracy of butterfly : 39.0%
Accuracy of camel : 37.0%
Accuracy of can : 51.0%
Accuracy of castle : 64.0%
Accuracy of caterpillar : 45.0%
Accuracy of cattle : 35.0%
Accuracy of chair : 70.0%
Accuracy of chimpanzee : 61.0%
Accuracy of clock : 35.0%
Accuracy of cloud : 65.0%
Accuracy of cockroach : 70.0%
Accuracy of couch : 35.0%
Accuracy of crab : 45.0%
Accuracy of crocodile : 30.0%
Accuracy of cup : 58.0%
Accuracy of dinosaur : 46.0%
Accuracy of dolphin : 45.0%
Accuracy of elephant : 36.0%
Accuracy of flatfish : 38.0%
Accuracy of forest 